In [23]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

In [24]:
class AgentState(TypedDict):
    first_number:int
    operation:str
    second_number:int
    final_number:int

In [25]:
from langchain_core.messages.ai import subtract_usage


def add(state:AgentState) -> AgentState:
    """This function calculates the sum of the two given numbers"""
    state['final_number']=state['first_number']+state['second_number']
    return state

def sub(state:AgentState):
    """This function calculates the difference of the two given numbers"""
    state['final_number']=state['first_number']-state['second_number']
    return state

def condition(state:AgentState) -> AgentState:
    """This node will decide where the program would go in execution"""
    if (state['operation']=="+"): return "addition_edge"
    if (state['operation']=="-"): return "subtract_edge"

In [26]:
graph=StateGraph(AgentState)
graph.add_node("adder", add)
graph.add_node("subtractor", sub)
graph.add_node("router", lambda state:state)
graph.add_edge(START, "router")
graph.add_conditional_edges("router", condition, {
    "addition_edge":"adder", "subtract_edge":"subtractor"
})
graph.add_edge("adder", END)
graph.add_edge("subtractor", END)
app=graph.compile()

In [30]:
results=app.invoke({'first_number':4, 'operation':"-", 'second_number':7})
results['final_number']

-3